# Using Tools


In [15]:
import instructor
import requests
import json
from pydantic import BaseModel, Field
from instructor import OpenAISchema
from openai import OpenAI

#Only for Jupyter Notebooks
import nest_asyncio
nest_asyncio.apply()

#--------------------------------------------------------
# Initialize the OpenAI client for Ollama 
#--------------------------------------------------------
client = OpenAI(
    base_url="http://localhost:11434/v1/", 
    api_key=""
)
client = instructor.patch(client)

#--------------------------------------------------------
# Tool response model 
#--------------------------------------------------------
class WeatherResponse(BaseModel):
    temperature: float = Field(description="The current temperature in celsius for the given location.")
    response: str = Field(description="A natural language response to the user's question.")

#----------------------------------------------------------
# Tool & OpenAISchema
#----------------------------------------------------------
class GetWeather(OpenAISchema):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

    def run(self) -> dict:
        """Fetch weather info from Open-Meteo"""
        response = requests.get(
            f"https://api.open-meteo.com/v1/forecast"
            f"?latitude={self.latitude}&longitude={self.longitude}"
            f"&current=temperature_2m,wind_speed_10m"
        )
        data = response.json()
        return data["current"]

#--------------------------------------------------------
# Tool schema
#--------------------------------------------------------
GetWeather.openai_schema["strict"] = True
#GetWeather.openai_schema["max_retries"] = 3
tools = [{
    "type": "function", 
    "function":GetWeather.openai_schema
}]

print(GetWeather.openai_schema)

{'name': 'GetWeather', 'description': 'Correctly extracted `GetWeather` with all the required parameters with correct types', 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'}, 'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}}, 'required': ['latitude', 'longitude'], 'type': 'object'}, 'strict': True}


In [16]:
model="llama3.1:8b"
messages=[            
    {"role": "system",  "content": "You are a helpful weather assistant."},
    {"role": "user",    "content": "What's the weather like now in Tainan, Taiwan now ?"},
]

#---------------------------------------------------------------------------------------------
# 這是 OpenAI API 官方標準的端點 ===> 用於創建聊天對話的補全 (chat completion)。
#---------------------------------------------------------------------------------------------
# 1. 根據 messages 和 tools，模型決定是要直接生成文字回覆，還是要呼叫工具 (tool call)。
#    當模型決定呼叫工具時，它的回覆會包含工具名稱和參數，但不會執行工具。
#---------------------------------------------------------------------------------------------
completion2 = client.chat.completions.create(
    model=model,    
    messages=messages,     
    temperature=0, 
    tools=tools,      
)
#print(f"\nchat completion ===>\n", json.dumps(completion2.model_dump(), indent=2))   

#-------------------------------------------------------------------
# Retries = 3 and async
#-------------------------------------------------------------------
import inspect
import time
import asyncio

async def call_with_retry(fn, retries=3, delay=1):
    for attempt in range(retries):
        try:
            result = fn()
            if inspect.isawaitable(result):
                result = await result               
                 
            return result
        except Exception as e:
            print(f"[Retry] 第 {attempt+1} 次失敗: {e}")
            if attempt < retries - 1:
                # 若在 async context 中，使用 asyncio.sleep
                if asyncio.get_event_loop().is_running():
                    await asyncio.sleep(delay)
                else:
                    time.sleep(delay)
            else:
                raise

#----------------------------------------------------------------------------
# 2. 如果 LLM 說要 Call Tool
#----------------------------------------------------------------------------
tool_calls = completion2.choices[0].message.tool_calls
if tool_calls:  
    for tool_call in tool_calls:
        # 3. 取出 Tool 的 name 和 參數    
        # 4. 把 LLM 回覆加入到 messages 列表中。使 messages 現在包含了使用者發問和 LLM 的回覆
        name = tool_call.function.name    
        args = json.loads(tool_call.function.arguments)        
        messages.append(completion2.choices[0].message)    
        print(f"\nTool ===> {name} with args: {args}, \nmessage ===> ", completion2.choices[0].message)   
          
        # 5. Tool Calling
        result = None
        if name == "GetWeather":     
            result = await call_with_retry(lambda: GetWeather(**args).run(), retries=3)    
            messages.append(
                {"role": "tool", "tool_call_id": tool_call.id, "content": json.dumps(result)}
            )  
            print(f"\nTool Calling 結果 ===> {result}")           
            


Tool ===> GetWeather with args: {'latitude': 22.98, 'longitude': 120.21}, 
message ===>  ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_frp1tcjq', function=Function(arguments='{"latitude":22.98,"longitude":120.21}', name='GetWeather'), type='function', index=0)])

Tool Calling 結果 ===> {'time': '2025-09-07T04:15', 'interval': 900, 'temperature_2m': 31.5, 'wind_speed_10m': 20.3}


In [17]:
#----------------------------------------------------------------------------
# 6. 最後讓 LLM 根據工具回傳的內容，生成你想要的自然語言回應。
#    而且回傳格式會符合你定義的 WeatherResponse Pydantic schema
#----------------------------------------------------------------------------
completion = client.chat.completions.parse(
    model=model,
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

final_response = completion.choices[0].message.parsed
print("\nWeatherResponse Model 定義的 temperature ===> ", final_response.temperature)
print("WeatherResponse Model 定義的 response ===> ",final_response.response)



WeatherResponse Model 定義的 temperature ===>  31.5
WeatherResponse Model 定義的 response ===>  Currently in Tainan, Taiwan it is 31.5°C (approximately 88.7°F) with a wind speed of about 20.3 km/h (12.6 mph). Please note that this information is accurate as of the provided timestamp (2025-09-07T04:15) and may have changed since then.
